In [1]:
!pip install datasets lm_eval py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.7 MB/s eta 

In [2]:
from tqdm.notebook import tqdm

from datasets import load_dataset
import torch
from torch.utils.data import DataLoader

from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
)
from transformers import default_data_collator, Trainer, TrainingArguments,AutoModelForCausalLM, AutoTokenizer


# Loading the model

In [3]:
model_name = "Shahrukh0/short-llama-3.2-3B-final-5l"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/453M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [4]:
# models architecture
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-22): 23 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# referencing https://github.com/meta-llama/llama-recipes/blob/main/recipes/finetuning/huggingface_trainer/peft_finetuning.ipynb
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow's afternoon?
B: I'm pretty sure I am. What's up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we've discussed it many times. I think he's ready now.
B: That's good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he'd name it after his dead hamster - Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100, use_cache=True)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this dialog:
A: Hi Tom, are you busy tomorrow's afternoon?
B: I'm pretty sure I am. What's up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we've discussed it many times. I think he's ready now.
B: That's good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he'd name it after his dead hamster - Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
A)k  a  a  p  p  p  u  p  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y  y 


# loading the dataset

In [6]:
import logging
logging.basicConfig(level=logging.INFO)

In [7]:
def get_preprocessed_samsum():
    dataset = load_dataset("samsum", split="train")

    prompt = (
        f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n"
    )

    def apply_prompt_template(sample):
        return {
            "prompt": prompt.format(dialog=sample["dialogue"]),
            "summary": sample["summary"],
        }

    dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))

    def tokenize_add_label(sample):
        prompt = tokenizer.encode(tokenizer.bos_token + sample["prompt"], add_special_tokens=False)
        summary = tokenizer.encode(sample["summary"] +  tokenizer.eos_token, add_special_tokens=False)
        sample = {
            "input_ids": prompt + summary,
            "attention_mask" : [1] * (len(prompt) + len(summary)),
            "labels": [-100] * len(prompt) + summary,
            }

        return sample

    dataset = dataset.map(tokenize_add_label, remove_columns=list(dataset.features))

    return dataset

In [8]:
dataset = get_preprocessed_samsum()
print(f"Dataset loaded with {len(dataset)} examples")

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Dataset loaded with 14732 examples


In [14]:
model.train()

def create_peft_config(model):
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 1,884,160 || all params: 2,711,286,784 || trainable%: 0.0695


In [15]:
model.config.use_cache = False

In [16]:
output_dir = "tmp/"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-6,
    'num_train_epochs': 1,
    'per_device_train_batch_size': 1,
    'gradient_checkpointing': True,
}

In [17]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    # logging strategies
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="no",
    fp16=True,
    report_to="none",  # 👈 Enable wandb logging
    optim="adamw_torch_fused",
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=default_data_collator,
    callbacks=[],
)


In [18]:
torch.cuda.empty_cache()

In [19]:
# Start training
print("Starting training...")
trainer.train()
print("Training completed!")

Starting training...


Step,Training Loss
100,2.943200
200,2.970800
300,2.977700
400,2.777600
500,2.787500
600,2.594900
700,2.479700
800,2.345600
900,2.363700
1000,2.302200


Training completed!


In [20]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Summarize this dialog:
A: Hi Tom, are you busy tomorrow's afternoon?
B: I'm pretty sure I am. What's up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we've discussed it many times. I think he's ready now.
B: That's good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he'd name it after his dead hamster - Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
Tom's son's getting a puppy. Tom will go with him to the animal shelter. Tom will get him one of the little dogs.


In [39]:
from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM

def evaluate_loaded_model(model, tokenizer, tasks, num_fewshot=0):
    """
    Evaluates an already-loaded Hugging Face model using lm-eval harness
    
    Args:
    - model: Loaded Hugging Face model
    - tokenizer: Loaded Hugging Face tokenizer
    - tasks: List of tasks to evaluate
    - num_fewshot: Number of few-shot examples to use
    
    Returns:
    - Dictionary of metrics
    """
    # Create model wrapper for lm-eval
    lm = HFLM(
        pretrained=model,
        tokenizer=tokenizer,
        device="cuda",
        batch_size=1  # Keep batch_size=1 to match original hyperparameters
    )
    
    # Run evaluation with same parameters
    results = evaluator.simple_evaluate(
        model=lm,
        tasks=tasks,
        num_fewshot=num_fewshot,
        limit=1000,
        bootstrap_iters=10,
        log_samples=False
    )
    
    return results['results']

In [40]:

tasks = ["hellaswag", "mmlu", "boolq", "lambada", "arc_easy"]
metrics = evaluate_loaded_model(model, tokenizer, tasks, num_fewshot=0)
print(metrics)

README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

mmlu_no_train.py:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

data.tar:   0%|          | 0.00/166M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

bootstrapping for stddev: perplexity


100%|██████████| 1/1 [00:00<00:00, 167.72it/s]


bootstrapping for stddev: perplexity


100%|██████████| 1/1 [00:00<00:00, 165.42it/s]


{'arc_easy': {'alias': 'arc_easy', 'acc,none': 0.589, 'acc_stderr,none': 0.015566673418599275, 'acc_norm,none': 0.583, 'acc_norm_stderr,none': 0.015599819048769618}, 'boolq': {'alias': 'boolq', 'acc,none': 0.644, 'acc_stderr,none': 0.015149042659306626}, 'hellaswag': {'alias': 'hellaswag', 'acc,none': 0.434, 'acc_stderr,none': 0.015680876566375058, 'acc_norm,none': 0.555, 'acc_norm_stderr,none': 0.01572330188676094}, 'lambada_openai': {'alias': 'lambada_openai', 'perplexity,none': 17.59226011110373, 'perplexity_stderr,none': 1.558970635382, 'acc,none': 0.505, 'acc_stderr,none': 0.015818508944436656}, 'lambada_standard': {'alias': 'lambada_standard', 'perplexity,none': 22.901967522173123, 'perplexity_stderr,none': 2.198944687386243, 'acc,none': 0.445, 'acc_stderr,none': 0.01572330188676094}, 'mmlu': {'acc,none': 0.5298341723423157, 'acc_stderr,none': 0.004075316349180567, 'alias': 'mmlu'}, 'mmlu_humanities': {'acc,none': 0.48165907456245505, 'acc_stderr,none': 0.007235422795524326, 'ali

In [41]:
# Save adapter and base model
model.save_pretrained("short-llama-3.2-3B-final-5l-fine-tuned")
tokenizer.save_pretrained("short-llama-3.2-3B-final-5l-fine-tuned")

# Push to Hub
model.push_to_hub("Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned")
tokenizer.push_to_hub("Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned")

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned/commit/5b108cd1a05230b013a914c64a188a6ba67ce223', commit_message='Upload tokenizer', commit_description='', oid='5b108cd1a05230b013a914c64a188a6ba67ce223', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned'), pr_revision=None, pr_num=None)

In [42]:
# Merge LoRA adapter with base model
merged_model = model.merge_and_unload()

# Save merged model
merged_model.save_pretrained("short-llama-3.2-3B-final-5l-fine-tuned")
tokenizer.save_pretrained("short-llama-3.2-3B-final-5l-fine-tuned")

# Push to Hub
merged_model.push_to_hub("Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned")
tokenizer.push_to_hub("Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned/commit/1268c5c19d47a319604fa148cf239eec3fc2c2a1', commit_message='Upload tokenizer', commit_description='', oid='1268c5c19d47a319604fa148cf239eec3fc2c2a1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Shahrukh0/short-llama-3.2-3B-final-5l-fine-tuned'), pr_revision=None, pr_num=None)